In [2]:
import matplotlib
matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180

import sys
sys.path.append("/home/cat/code/widefield/") # go to parent dir



import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import glob2

from sklearn import linear_model
from scipy import stats
import numpy as np
import matplotlib.patches as mpatches


# visualize results module
from visualize import visualize_svm as Visualize

import scipy

Autosaving every 180 seconds


In [40]:

def find_code04_starts2(abscodes, abstimes, abspositions, lockout=3.0):
    vals = []
    
    if type(abscodes[0])==np.int64:
        vals = abscodes
    else:
        for k in range(abscodes.shape[0]):
            vals.append(np.int(abscodes[k].decode()))
        vals=np.array(vals)
        
    #   
    threshold = 40
    idx04 = np.where(np.logical_and(vals==4, np.abs(abspositions)>=threshold))[0]
    idx02 = np.where(np.logical_and(vals==2, np.abs(abspositions)>=threshold))[0]
    
    # figure out the first place where 04 code is registered;
    starts_04 = []
    starts_04_idx = []
    for k in range(1, abstimes[idx04].shape[0], 1):
        if (abstimes[idx04][k]-abstimes[idx04][k-1])>lockout:
            starts_04.append(abstimes[idx04[k]])
            starts_04_idx.append(idx04[k])
            
    starts_04 = np.array(starts_04)
    starts_04_idx = np.array(starts_04_idx)

    # starts 02 bad performance
    starts_02 = []
    starts_02_idx = []
    for k in range(1, abstimes[idx02].shape[0], 1):
        if (abstimes[idx02][k]-abstimes[idx02][k-1])>lockout:
            starts_02.append(abstimes[idx02][k])
            starts_02_idx.append(idx02[k])

    starts_02 = np.array(starts_02)
    starts_02_idx = np.array(starts_02_idx)
    
    return starts_04, starts_04_idx, starts_02, starts_02_idx


def load_lever_data(root_dir):
    temp = root_dir + '*abstimes.npy'
    #print ("TRYING TO LOAD ABSTIMES: ", temp)
    print (temp)
    try:
        fname = glob2.glob(temp)[0]
    except:
        print (glob2.glob(temp))
        return [], [], []

    abstimes = np.load(fname)
    fname = glob2.glob(root_dir + '*abspositions.npy')[0]
    abspositions = np.load(fname)
    fname = glob2.glob(root_dir + '*abscodes.npy')[0]
    abscodes = np.load(fname)

    return abstimes, abspositions, abscodes






In [107]:
################################################
################################################
################################################
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    root_dir = '/media/cat/4TBSSD/yuki/'
    #animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
    animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
    ctr=0
    #fig=plt.figure()
    fig1, axes1 = plt.subplots(nrows=3, ncols=2)
    fig2, axes2 = plt.subplots(nrows=3, ncols=2)
    fig3, axes3 = plt.subplots(nrows=3, ncols=2)

    #
    for ctr_a, animal_id in enumerate(animal_ids):
        ax=plt.subplot(3,2,ctr+1)
        sessions = np.loadtxt(os.path.join(root_dir,
                                          animal_id,
                                          'tif_files',
                                          'sessions.txt'), dtype='str')

        sessions = np.load(os.path.join(root_dir,
                                        animal_id,
                                        'tif_files.npy'))
        #
        n_trials_04 =[]
        n_trials_02 = []
        for session in sessions:
            session = os.path.split(session)[1].replace('.tif','')
            try:
                if False:
                    # 
                    abstimes, abspositions, abscodes = load_lever_data(os.path.join(root_dir,
                                                                                animal_id,
                                                                                'tif_files',
                                                                                session,
                                                                                   session+"_"))

                    #
                    (starts_04, starts_04_idx,starts_02, starts_02_idx) = find_code04_starts2(abscodes, abstimes, abspositions)

                else:

                    starts_04 = np.loadtxt(os.path.join(root_dir,
                                      animal_id,
                                      'tif_files',
                                      session,
                                      'rewarded_times.txt'))

                    starts_02 = np.loadtxt(os.path.join(root_dir,
                                      animal_id,
                                      'tif_files',
                                      session,
                                      'nonrewarded_times.txt'))

                # these are probably errors, need to delete > 200 trials per session;
                if starts_04.shape[0]>200: #or starts_02.shape[0]>300:
                    continue
                n_trials_04.append(starts_04.shape[0])
                n_trials_02.append(starts_02.shape[0])
            except:
                pass
        
        n_trials_04 = n_trials_04[5:]
        n_trials_02 = n_trials_02[5:]
        
        if True:
            t = np.arange(len(n_trials_04))
            axes1[ctr_a//2,ctr_a%2].scatter(t,n_trials_04,c='black',
                        s=100,
                       alpha=.25)
            temp2 = np.poly1d(np.polyfit(t, n_trials_04, 1))
            axes1[ctr_a//2,ctr_a%2].plot(t,temp2(t), '--',c= 'black',
                    alpha=1,
                    linewidth=3)
            corr = scipy.stats.pearsonr(t,n_trials_04)
            print ("rewarded lever pulls: ", animal_id, "corr: ", corr)
            
            axes1[ctr_a//2,ctr_a%2].set_xlim(t[0],t[-1])
            #axes1[ctr_a//2,ctr_a%2].set_ylim(0,min(6,np.max(n_trials_04)*1.1))
            axes1[ctr_a//2,ctr_a%2].set_xticks([])

        if True:
        
            t = np.arange(len(n_trials_02))
            axes2[ctr_a//2,ctr_a%2].scatter(t,n_trials_02,c='red',
                        s=100,
                       alpha=.25)
            temp2 = np.poly1d(np.polyfit(t, n_trials_02, 1))
            axes2[ctr_a//2,ctr_a%2].plot(t,temp2(t), '--',c= 'red',
                    alpha=1,
                    linewidth=3)    
            plt.xlim(t[0],t[-1])
            plt.ylim(0,min(6,np.max(n_trials_02)*1.1))
            axes1[ctr_a//2,ctr_a%2].set_xlim(t[0],t[-1])
            #axes1[ctr_a//2,ctr_a%2].set_ylim(0,min(6,np.max(n_trials_04)*1.1))
            axes1[ctr_a//2,ctr_a%2].set_xticks([])
            
        # 
        if False:
            ratios = np.array(n_trials_04)/np.array(n_trials_02)
            ratios[ratios == np.inf] = np.nan
            idx = np.where(np.isnan(ratios)==False)[0]
            ratios = ratios[idx]
            
            #idx2 = np.where(ratios>5)[0]
            #ratios = np.delete(ratios,idx2)
            
            # print (ratios)
            t = np.arange(len(ratios))

            axes3[ctr_a//2,ctr_a%2].scatter(t,ratios,c='green',
                        s=100,
                       alpha=.25)

            temp2 = np.poly1d(np.polyfit(t, ratios, 1))
            # 
            axes3[ctr_a//2,ctr_a%2].plot(t,temp2(t), '--',c= 'green',
                    alpha=1,
                    linewidth=3)    
            plt.xlim(t[0],t[-1])
            plt.ylim(0,min(6,np.max(ratios)*1.1))
            axes1[ctr_a//2,ctr_a%2].set_xlim(t[0],t[-1])
            #axes1[ctr_a//2,ctr_a%2].set_ylim(0,min(6,np.max(n_trials_04)*1.1))
            axes1[ctr_a//2,ctr_a%2].set_xticks([])            
            corr = scipy.stats.pearsonr(t,ratios)
            #print ("ratio: ", animal_id, "corr: ", corr)
            

        #    
        ctr+=1

        #if np.max(n_trials)>100:
        #    plt.ylim(0,200)
        #else:
        #    plt.ylim(0,100)
        #plt.legend(animal_id+)
        #plt.yticks([])

    if True:
        fig1.savefig('/home/cat/n_trials_longitudinal.svg')
        plt.close()
    else:
        plt.show()
    

rewarded lever pulls:  IA1 corr:  (-0.250106289580812, 0.04804765614761055)
rewarded lever pulls:  IA2 corr:  (0.38619221115218794, 0.01999455618611675)
rewarded lever pulls:  IA3 corr:  (0.6548777899619983, 1.0941816906562287e-05)
rewarded lever pulls:  IJ1 corr:  (-0.374520345326465, 0.022385893517622046)
rewarded lever pulls:  IJ2 corr:  (0.17952691356310058, 0.2876881937476088)
rewarded lever pulls:  AQ2 corr:  (0.6785896541289496, 6.32023973894543e-15)


In [38]:
######################################################
######################################################
######################################################
fname = '/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Mar15_Week2_30Hz/AQ2pm_Mar15_Week2_30Hz'

# 
abstimes, abspositions, abscodes = load_lever_data(os.path.join(os.path.split(fname)[0]+'/'))

#
(starts_04, starts_04_idx,starts_02, starts_02_idx) = find_code04_starts2(abscodes, abstimes, abspositions)

            
data = np.load(fname+'_locs44threshold.npy')
codes = np.load(fname+'_code44threshold.npy')
print (data.shape, codes.shape)


code = b'04'
idx = np.where(codes==code)[0]
print ("locs 44 # of 04 codes", idx.shape)
plt.plot(data, np.arange(data.shape[0]))
plt.scatter(data[idx], idx,
           s=100,c='red')

# 
abscodes = np.load(fname+'_abscodes.npy')
idx = np.where(abscodes==code)[0]

abspositions = np.load(fname+'_abspositions.npy')
abstimes = np.load(fname+'_abstimes.npy')
plt.plot(abstimes, abspositions)
plt.scatter(abstimes[idx], abspositions[idx])

temp1 = np.loadtxt(os.path.split(fname)[0]+'/rewarded_times.txt')
print ("# of erwarded times: ", temp1.shape)
plt.scatter(temp1, temp1*0+10,
           c='magenta',
           s=200)

print ("starts_04: ", starts_04.shape)
plt.scatter(starts_04, starts_04*0+20,
           c='pink',
           s=200)
 
            
plt.show()
     #  4TBSSD/yuki/AQ2/tif_files/AQ2pm_Mar15_Week2_30Hz/rewarded_times.txt

/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Mar15_Week2_30Hz/*abstimes.npy


<ipython-input-37-0c35672e529e>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vals.append(np.int(abscodes[k].decode()))


(283,) (283,)
locs 44 # of 04 codes (145,)
# of erwarded times:  (145,)
starts_04:  (83,)


In [43]:
data = np.load('/media/cat/4TBSSD/yuki/IA2/tif_files.npy')
print (data)

['/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb1_30Hz/IA2pm_Feb1_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb2_30Hz/IA2pm_Feb2_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb3_30Hz/IA2pm_Feb3_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb4_30Hz/IA2pm_Feb4_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb5_30Hz/IA2pm_Feb5_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb9_30Hz/IA2pm_Feb9_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb10_30Hz/IA2pm_Feb10_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb11_30Hz/IA2pm_Feb11_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb12_30Hz/IA2pm_Feb12_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb15_30Hz/IA2pm_Feb15_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb16_30Hz/IA2pm_Feb16_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb17_30Hz/IA2pm_Feb17_30Hz.tif'
 '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb18_30Hz/IA2pm_Feb18_30Hz.tif